In [ ]:
!wget https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_Dataset.zip
!wget https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_text.zip
!unzip Flickr8k_Dataset.zip -d all_images
!unzip Flickr8k_text.zip -d all_captions

: 

In [10]:
import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
import json
from time import time
import pickle

import nltk
from nltk.corpus import stopwords

from torchvision.models import resnet50, ResNet50_Weights
import torch

image_path = "../all_images/Flicker8k_Dataset/"

In [11]:
def clean_text(text):
  text = text.lower()
  text = re.sub("[^a-z]+", " ", text)
  text = text.split()
  text = [s for s in text if len(s)> 1]
  text = " ".join(text)
  return text
clean_text("Tes tç% awdoujç TçG")

'tes awdouj'

In [12]:
train_file = open(train_file_path, "r").read()
train = [row.split(".")[0] for row in train_file.split("\n")[:-1]]
print(len(train))
print(train[:5])


test_file = open(test_file_path, 'r').read()
test = [row.split(".")[0] for row in test_file.split("\n")[:-1]]
print(len(test))
print(test[:5])

6000
['2513260012_03d33305cf', '2903617548_d3e38d7f88', '3338291921_fe7ae0c8f8', '488416045_1c6d903fe0', '2644326817_8f45080b87']
1000
['3385593926_d3e9c21170', '2677656448_6b7e7702af', '311146855_0b65fdb169', '1258913059_07c613f7ff', '241347760_d44c8d3a01']


In [163]:
caption_file = open(captions_path, "r").read()
captions = {}
all_captions = []

word_to_idx = {}
idx_to_word = {}

for row in caption_file.split("\n"):
    multiple_captions = []
    key = row.split("#")[0].split(".")[0]
    caption = row.split("#")[-1].split("\t")[-1]
    caption = caption.replace('.', '')
    caption = clean_text(caption)
    caption = "<start> " + caption + " <end>"
    all_captions.append(caption)
    if  key not in captions:
        captions[key] = []
        captions[key].append(caption)
    else:
        captions[key].append(caption)

print(len(captions))
print(captions["2903617548_d3e38d7f88"])

8093
['<start> little baby plays croquet <end>', '<start> little girl plays croquet next to truck <end>', '<start> the child is playing croquette by the truck <end>', '<start> the kid is in front of car with put and ball <end>', '<start> the little boy is playing with croquet hammer and ball beside the car <end>']


In [174]:
all_words = ""
for word in captions.values():
    all_words = all_words + " " + " ".join(word)
all_words = all_words[1:]
all_words = all_words.split(" ")[1:]
print(len(set(all_words)))
#print(set(all_words))
all_words = list(set(all_words))
print(len(all_words[1:]))
i = 0
for word in all_words[1:]:
    #print(word)
    word_to_idx[word] = i
    idx_to_word[i] = word
    i+=1

print(word_to_idx["<start>"])
print(len(idx_to_word))
print(idx_to_word[4795])

8426
8425
4795
8425
<start>


In [5]:
#model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
#model = models.resnet152(pretrained=True)
#newmodel = torch.nn.Sequential(*(list(model.children())[:-1]))
model = resnet50(weights=None)
model = torch.nn.Sequential(*(list(model.children())[:-1]))

In [6]:
import io

import requests
import torchvision.transforms as T

from PIL import Image

preprocess = T.Compose([
   T.Resize(256),
   T.CenterCrop(224),
   T.ToTensor()
])

"""T.Normalize(
       mean=[0.485, 0.456, 0.406],
       std=[0.229, 0.224, 0.225]
)"""

def preprocess_img(img):
    img = Image.open(img, 'r')
    img = preprocess(img)
    return img

def encode_image(img):
    img = preprocess_img(img)
    if len(img.shape) == 3:
        img = img[None, :]
    feature_vector = model(img)
    feature_vector = feature_vector.reshape((-1,))
    return feature_vector

print(encode_image(image_path + "1000268201_693b08cb0e.jpg").shape)


torch.Size([2048])


In [9]:
start_t = time()
encoding_train = {}

for ix, img_id in enumerate(train):
    img_path = image_path + img_id + ".jpg"
    #encoding_train[img_id] = encode_image(img_path)

    if ix%100==0:
        print("Encoding in Progress Time step %d"%ix)
end_t = time()
print("Total Time Taken: ", end_t-start_t)

with open("saved_encoded_train_features.pkl", "wb") as f:
    pickle.dump(encoding_train, f)

Encoding in Progress Time step 0
Encoding in Progress Time step 100
Encoding in Progress Time step 200
Encoding in Progress Time step 300
Encoding in Progress Time step 400
Encoding in Progress Time step 500
Encoding in Progress Time step 600
Encoding in Progress Time step 700
Encoding in Progress Time step 800
Encoding in Progress Time step 900
Encoding in Progress Time step 1000
Encoding in Progress Time step 1100
Encoding in Progress Time step 1200
Encoding in Progress Time step 1300
Encoding in Progress Time step 1400
Encoding in Progress Time step 1500
Encoding in Progress Time step 1600
Encoding in Progress Time step 1700
Encoding in Progress Time step 1800
Encoding in Progress Time step 1900
Encoding in Progress Time step 2000
Encoding in Progress Time step 2100
Encoding in Progress Time step 2200
Encoding in Progress Time step 2300
Encoding in Progress Time step 2400
Encoding in Progress Time step 2500
Encoding in Progress Time step 2600
Encoding in Progress Time step 2700
Enco

In [166]:
import collections

counter = collections.Counter(all_words)
dic = dict(counter)

threshold_value = 10
sorted_dic = sorted(dic.items(), reverse=True, key=lambda x: x[1])
sorted_dic = [x for x in sorted_dic if x[1] > threshold_value]

all_words = [x[0] for x in sorted_dic]
print(len(all_words))

0


In [199]:
max_len = 0
for key in captions.keys():
    for cap in captions[key]:
        max_len = max(max_len, len(cap.split()))

print(max_len)

35


In [200]:
len(all_words)

8426

In [201]:
f = open("../data/glove.6B.50d.txt", encoding='utf8')
embedding_index = {}

for line in f:
    values = line.split()
    word = values[0]
    vector = np.asarray(values[1:], dtype='float32')
    embedding_index[word] = vector

f.close()

print(len(embedding_index))

def get_embedding_matrix():
    emb_dim = 50
    embedding_matrix = np.zeros((len(all_words[1:])+1, emb_dim))
    print(embedding_matrix.shape, len(word_to_idx))
    for word, idx in word_to_idx.items():
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[idx] = embedding_vector
    
    return embedding_matrix

embedding_matrix = get_embedding_matrix()
print(embedding_matrix.shape)

400000
(8426, 50) 8425
(8426, 50)


In [202]:
import torch.nn as nn
from torch.autograd import Variable
from torchsummary import summary

class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()
        self.decoder = nn.Linear(2040, 1020)
        self.lstm = nn.LSTM(1020, 25)
        self.label = nn.Linear(100, 100)

    def forward(self, input):
        
        return self.label(final_hidden_state[-1])

In [209]:
model(torch.zeros(1,3,32,100)).shape

torch.Size([1, 2048, 1, 1])

In [203]:
lstm = LSTM()
lstm

LSTM(
  (lstm): LSTM(2040, 100)
  (label): Linear(in_features=100, out_features=100, bias=True)
)